In [1]:
import pandas as pd
from config import *
from filtering import formatDF, discretizarDatos

df = pd.read_csv(all)

data = formatDF(df)
data.loc[data["vendedor"].isna(), "vendedor"] = data.loc[data["vendedor"].isna(), "tipoVendedor"]
data = discretizarDatos(data)

# drop columns: activo, URL, file, tipoVendedor, ciudad
data = data.drop(['activo', 'URL', 'file', 'tipoVendedor','ciudad'], axis=1)
data = data.loc[data["tipoPropiedad"] != 3]
data['barrio'] = data['barrio'].astype('category').cat.codes

C:\Users\benja\AppData\Local\Temp\ipykernel_14680\3599382349.py:5: DtypeWarning: Columns (13,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(all)
C:\Users\benja\AppData\Local\Temp\ipykernel_14680\3599382349.py:7: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  data = formatDF(df)


In [2]:
import joblib
from sklearn.preprocessing import StandardScaler

features = ['terrenoEdificado', 'coordX', 'coordY', 'barrio']
data_for_clustering = data[features]

scaler_kmeans = StandardScaler()
scaled_data = scaler_kmeans.fit_transform(data_for_clustering)

kmeans = joblib.load(kmeans_model)
data['clusterKM'] = kmeans.predict(scaled_data)

In [7]:
import joblib
from sklearn.model_selection import train_test_split

ensamble = joblib.load(ensamble_precio_model)

features = ['terrenoEdificado', 'comisariaCercana', 'transporteCercano', 
            'saludCercana', 'coordX', 'coordY', 'clusterKM', 'barrio']
data_for_train = data[features]
target_column = 'precioUSD'
target = data[target_column]

scaler_ensamble = StandardScaler()
scaled_data = scaler_ensamble.fit_transform(data_for_train)

X_train, X_test, y_train, y_test = train_test_split(data_for_train, target, train_size=0.99, random_state=33)

In [8]:
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

X_test_scaled = scaler_ensamble.transform(X_test)
all_data_predictions = ensamble.predict(X_test_scaled)

ensemble_r2 = r2_score(y_test, all_data_predictions)
ensemble_rmse = np.sqrt(mean_squared_error(y_test, all_data_predictions))